In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from random import randint

In [14]:
#colunas que escolhi para dados escolares
col_escol = ['CODMEC','ANO','TIPOESC','DEP','Qtde_Docentes','NUM_FUNC','MERENDA','CLFUND',
            'ALFUND','TOTALEJA','TOT_EE','TOTAL_ALUNOS','CL_CRECHE','AL_CRECHE','CL_PRE','AL_PRE','CLEM','ALEM']
col_rendi = ['CODMEC','EFABAND9ANOS8S','EFABAND9ANOS9S','EFABAND7S','EFABAND8S','TOTALEF_ABAND','TOTALEF_APROV',
             'TOTALEF_REPROV','TOTAL_EM_ABAND','TOTAL_EM_REPROV','CONCLUINTES_EM','EPABAND','EPREPROV']
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1', usecols = col_escol)
rendimento = pd.read_csv('RendimentoEscolar2000-2015.csv', encoding = 'latin1', usecols = col_rendi)
esc_rmsp = pd.read_csv('ESC2013_RMSP_CEM.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')
#como eu abri todas as colunas do esc_rmsp, agora vou retirar as que não desejo usar
col_rmsp = ['ID', 'LONGITUDE', 'LATITUDE', 'CODESCTX','NOMEESC', 'NOMEMUN','CODMUN', 'NOMDIST', 'CODDIST', 'COD_SC',
           'BAIRRO','CEP','END_ESC','NUM_ESC','DDD','TELEFONE','ENS_REG','REG_IN_CR','REG_IN_PE','REG_FU_8A',
           'REG_FU_9A','REG_MED','REG_MED_I','REG_MED_N','REG_MED_P','ENS_ESP','ESP_IN_CR','ESP_IN_PE',
           'ESP_FU_8A','ESP_FU_9A','ESP_MED','ESP_MED_I','ESP_MED_N','ESP_MED_P','ESP_EJA_F','ESP_EJA_M','EJA',
           'EJA_FUND','EJA_MED','EJA_PROJ']
esc_rmsp = esc_rmsp.drop(col_rmsp, axis = 1)

In [15]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(104292, 18)
Rendimento(14944, 13)
ESC_RMSP(10937, 96)
Nota(1243, 3)


In [16]:
#def t(x):
#    if x==0:
#        return None
#    else:
#        return x
#for c in dados_escolares.columns:
#    dados_escolares[c] = dados_escolares[c].apply(t)

In [25]:
#Agrupo por escolas
dados_escolares = dados_escolares.groupby('CODMEC').mean()
rendimento = rendimento.groupby('CODMEC').mean()
esc_rmsp = esc_rmsp.groupby('CODESC').mean()

In [26]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(690, 14)
Rendimento(504, 12)
ESC_RMSP(931, 20)
Nota(1241, 3)


In [27]:
#Apagando a classe desbalanceada
nota = nota[nota.Classificacao != 0]
#Em todos Dataframes temos mais escolas do que o nosso target
dados_escolares = dados_escolares.reset_index()[dados_escolares.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
rendimento = rendimento.reset_index()[rendimento.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
esc_rmsp = esc_rmsp.reset_index()[esc_rmsp.reset_index().CODESC.isin(nota['CODIGO DA ENTIDADE'])]

In [28]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(690, 15)
Rendimento(504, 13)
ESC_RMSP(931, 21)
Nota(1241, 3)


In [29]:
#apagando as linha com valor NaN
dados_escolares.dropna(inplace = True, how = 'any')
esc_rmsp.drop(esc_rmsp.columns[esc_rmsp.isnull().mean() > 0.15], axis=1, inplace = True)
esc_rmsp.dropna(how='any', axis = 0, inplace = True)

In [30]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(690, 15)
Rendimento(504, 13)
ESC_RMSP(931, 21)
Nota(1241, 3)


In [31]:
#Alterando o nome da coluna para o merge
nota.rename(columns = {'CODIGO DA ENTIDADE':'CODMEC'}, inplace = True)
#criando o merge com base na coluna CODMEC usando o dataframe target e os dados escolares
new_df = pd.merge(dados_escolares, nota, on = 'CODMEC')
new_df.drop(['CODMEC', 'ANO','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)

In [37]:
#criando o data e o target
data = new_df.drop('Classificacao', axis = 1)
target = new_df.Classificacao

In [134]:
lm = LinearRegression()
scaler = StandardScaler()
data1 = scaler.fit_transform(data)
kf = KFold(5, shuffle = True)
r = []
for train_i, test_i in kf.split(data):
    x_train, x_test = data1[train_i], data1[test_i]
    y_train, y_test = target[train_i], target[test_i]
    lm.fit(x_train, y_train)
    r.append(lm.score(x_test, y_test))
r.sort(reverse = True)
print('Kfolds results: ', r)

Kfolds results:  [0.4808406694753451, 0.4655680690228231, 0.3975419271105904, 0.3668001977695857, 0.33174411057519326]


In [40]:
def modelo_svm_kf(data, target, gamma, C, k):
    kf = KFold(n_splits = k)
    scores = []
    for g in gamma:
            for c in C:
                mean_kf = []
                for train_i, test_i in kf.split(data):
                    #separação dos dados em treino e teste
                    data_train, data_test = data.iloc[train_i], data.iloc[test_i]
                    target_train, target_test = target[train_i], target[test_i]
                    
                    #normalização dos dados
                    scaler = StandardScaler()
                    scaler.fit(data_train)
                    data_train = scaler.transform(data_train)
                    data_test = scaler.transform(data_test)
                    
                    #pca nos dados
                    pca = PCA(0.95)
                    pca.fit(data_train)
                    data_train = pca.transform(data_train)
                    data_test = pca.transform(data_test)
                    
                    #rodando o svm
                    svc = SVC(kernel = 'rbf', C=c, gamma = g)
                    svc.fit(data_train,target_train)
                    mean_kf.append(svc.score(data_test,target_test))
                                   
                #salvando a média do KFold para g e c dados
                scores.append(((sum(mean_kf)/len(mean_kf)), g, c))
                print('Iteração com gamma %f e c %f concluída' %(g, c))
    return scores

In [135]:
res = modelo_svm_kf(data, target, gamma = [0.01, 0.1, 1, 10, 100], C = [0.01, 0.1, 1, 10, 100], k = 5)
res.sort()
res[-5:]

Iteração com gamma 0.010000 e c 0.010000 concluída
Iteração com gamma 0.010000 e c 0.100000 concluída
Iteração com gamma 0.010000 e c 1.000000 concluída
Iteração com gamma 0.010000 e c 10.000000 concluída
Iteração com gamma 0.010000 e c 100.000000 concluída
Iteração com gamma 0.100000 e c 0.010000 concluída
Iteração com gamma 0.100000 e c 0.100000 concluída
Iteração com gamma 0.100000 e c 1.000000 concluída
Iteração com gamma 0.100000 e c 10.000000 concluída
Iteração com gamma 0.100000 e c 100.000000 concluída
Iteração com gamma 1.000000 e c 0.010000 concluída
Iteração com gamma 1.000000 e c 0.100000 concluída
Iteração com gamma 1.000000 e c 1.000000 concluída
Iteração com gamma 1.000000 e c 10.000000 concluída
Iteração com gamma 1.000000 e c 100.000000 concluída
Iteração com gamma 10.000000 e c 0.010000 concluída
Iteração com gamma 10.000000 e c 0.100000 concluída
Iteração com gamma 10.000000 e c 1.000000 concluída
Iteração com gamma 10.000000 e c 10.000000 concluída
Iteração com gamm

[(0.7130434782608696, 100, 0.01),
 (0.7130434782608696, 100, 0.1),
 (0.7130434782608696, 100, 1),
 (0.7159420289855072, 0.01, 100),
 (0.7246376811594203, 0.01, 1)]

In [167]:
params = {'kernel': ['rbf'], 'C':[0.1], 'gamma':[100]}
svr = SVR()
clf = GridSearchCV(estimator = svr, param_grid = params, cv = 5, return_train_score=True)
clf.fit(data, target)
print( 'Treino: ', clf.cv_results_['mean_train_score'])
print( 'Teste: ', clf.cv_results_['mean_test_score'])

Treino:  [0.14075827]
Teste:  [-0.22302813]


In [ ]:
svr = SVR(kernel = 'linear')
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.2)
svr.fit(x_train, y_train)
print( 'Treino: ', svr.score(x_train, y_train))
print( 'Teste: ', svr.score(x_test, y_test))